In [1]:
import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.linear_model import LinearRegression
import scikitplot as skplt

In [2]:
# Lendo os dados dos arquivos ja preparados.
all_files = glob.glob('./data_city' + '/*.csv')

df = []

for file in tqdm(all_files):
    df.append(pd.read_csv(file, encoding='latin-1', delimiter=';', names=[
         'condicoes',
         'idade',
         'municipioIBGE',
         'resultadoTeste',
         'sexo',
         'paladar',
         'fadiga',
         'olfato',
         'garganta',
         'respirar',
         'febre',
         'tosse',
         'diarreia',
         'coriza',
         'dispneia'
    ]))

def df_length(e):
    return e.size

df.sort(reverse=True, key=df_length)

df[0].head()

  0%|          | 0/1865 [00:00<?, ?it/s]

,condicoes,idade,municipioIBGE,resultadoTeste,sexo,paladar,fadiga,olfato,garganta,respirar,febre,tosse,diarreia,coriza,dispneia
0,0,31.0,5208707,1,0,0,0,0,0,0,1,1,0,1,0
1,0,90.0,5208707,0,0,0,0,0,0,0,1,1,0,0,0
2,0,44.0,5208707,0,1,0,0,0,0,0,1,1,0,0,0
3,0,51.0,5208707,1,1,0,0,0,0,0,0,1,0,0,0
4,0,59.0,5208707,1,1,0,0,0,0,0,1,0,0,0,0


In [3]:
# Função para separar em conjunto de treino e de teste.

def train_test_set(X, Y):
    return train_test_split(X, Y, test_size = 0.2)

In [4]:
# Função para separar input e output

def input_output_set(df):
    Y = df.iloc[:,3].values
    X = df.drop(['resultadoTeste', 'municipioIBGE'], axis=1).values
    return X, Y
    

In [21]:
# Função para normalizar os dados

def normalize(X):
    scaler = StandardScaler()
    return scaler.fit_transform(X)

In [16]:
# Funcão para balancear os dados

def balance_set(X, Y):
    balancer = RandomOverSampler()
    return balancer.fit_resample(X, Y)

In [23]:
# Função para executar a regressão linear

def linear_regression(df):
    X, Y = input_output_set(df)
    X = normalize(X)
    Xtrain, Xtest, Ytrain, Ytest = train_test_set(X, Y)
    
    regressor = LinearRegression()
    regressor.fit(Xtrain, Ytrain)
    Ypred = regressor.predict(Xtest)

    Ypred = [round(value) for value in Ypred]

    # Imprimindo a Acurácia
    accuracy = accuracy_score(Ytest, Ypred)
    print(f'Acurácia: {accuracy*100}%\nTamanho do conjunto:{len(X)}')

    # Imprimir matriz de confusão
    # skplt.metrics.plot_confusion_matrix(Ytest, Ypred, normalize=True)
    
    return accuracy*100

In [24]:
li = []

for data in df:
    if len(data.iloc[:,0]) < 100:
        break
    li.append(linear_regression(data))

Acurácia: 58.12835951300119%
Tamanho do conjunto:263244
Acurácia: 62.20332779639066%
Tamanho do conjunto:177595
Acurácia: 69.0227576974565%
Tamanho do conjunto:56022
Acurácia: 66.58602670795433%
Tamanho do conjunto:51668
Acurácia: 65.12800237224474%
Tamanho do conjunto:50584
Acurácia: 62.55203619909502%
Tamanho do conjunto:27622
Acurácia: 60.139464756879%
Tamanho do conjunto:26527
Acurácia: 76.51854871585813%
Tamanho do conjunto:24530
Acurácia: 58.55626326963906%
Tamanho do conjunto:23549
Acurácia: 70.12779552715655%
Tamanho do conjunto:21906
Acurácia: 66.84033098375728%
Tamanho do conjunto:16314
Acurácia: 61.435832274459976%
Tamanho do conjunto:15738
Acurácia: 66.16932685634976%
Tamanho do conjunto:14409
Acurácia: 83.55984217448487%
Tamanho do conjunto:11405
Acurácia: 70.41993539455468%
Tamanho do conjunto:10834
Acurácia: 59.21296296296297%
Tamanho do conjunto:10799
Acurácia: 58.09207403891789%
Tamanho do conjunto:10534
Acurácia: 62.76232308443143%
Tamanho do conjunto:10242
Acurácia: 